In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from tqdm import tqdm
from collections import defaultdict
import requests
import ipyparallel as ipp
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import math
import time
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn.metrics import r2_score
import logging
logging.getLogger("cmdstanpy").setLevel(logging.ERROR)
import itertools

In [21]:
def fetch_counts_for_year(year):
    counts_for_year = defaultdict(int)
    offset = 0
    limit = 10000
    while True:
        url = (
            f"https://data.cnra.ca.gov/api/3/action/datastore_search?"
            f"resource_id=bfa9f262-24a1-45bd-8dc8-138bc8107266"
            f"&q={year}&limit={limit}&offset={offset}"
        )
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            records = data['result']['records']
            for record in records:
                site_code = record.get('site_code')
                if record.get('gse_gwe') is None:
                    continue
                if site_code:
                    counts_for_year[site_code] += 1
            if len(records) < limit:
                break
            offset += limit
        else:
            print(f"Failed to fetch data for year {year}")
            break
    return counts_for_year

rc = ipp.Client()
dview = rc[:]

with dview.sync_imports():
    import requests
    from collections import defaultdict

dview.push({'fetch_counts_for_year': fetch_counts_for_year})

years = list(range(2000, 2025))

async_results = dview.map_async(fetch_counts_for_year, years)
results = async_results.get()

counts = defaultdict(lambda: defaultdict(int))
for year, year_counts in zip(years, results):
    for site_code, count in year_counts.items():
        counts[site_code][year] = count

obs_df = pd.DataFrame.from_dict(counts, orient='index')
obs_df = obs_df.fillna(0).astype(int)
obs_df.columns = [f'observations_{year}' for year in obs_df.columns]
obs_df = obs_df.reset_index().rename(columns={'index': 'site_code'})

importing requests on engine(s)
importing defaultdict from collections on engine(s)


In [22]:
columns_to_keep = ['site_code'] + [
    col for col in obs_df.columns 
    if col.startswith('observations_') and int(col.split('_')[1]) >= 2008
]

ov25 = obs_df[columns_to_keep]

In [23]:
observation_columns = [col for col in ov25.columns if col.startswith('observations_')]
ov25 = ov25[ov25[observation_columns].ge(25).all(axis=1)]

In [25]:
def fetch_all_records(site_code, year):
    all_records = []
    offset = 0
    limit = 1000
    while True:
        url = (
            f"https://data.cnra.ca.gov/api/3/action/datastore_search?"
            f"resource_id=bfa9f262-24a1-45bd-8dc8-138bc8107266"
            f"&q={site_code} {year}&limit={limit}&offset={offset}"
        )
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            records = data['result']['records']
            all_records.extend(records)
            if len(records) < limit:
                break
            offset += limit
        else:
            print(f"Failed to fetch data for {site_code} in {year}")
            break
    return all_records

def get_readings_for_site(site_code, years):
    all_readings = []
    for year in years:
        records = fetch_all_records(site_code, year)
        for record in records:
            reading = {
                'site_code': record.get('site_code'),
                'msmt_date': record.get('msmt_date'),
                'wlm_rpe': record.get('wlm_rpe'),
                'wlm_gse': record.get('wlm_gse'),
                'gwe': record.get('gwe')
            }
            all_readings.append(reading)
    return pd.DataFrame(all_readings)

In [26]:
rc = ipp.Client(timeout=1200)
dview = rc[:]

dview.push({
    'fetch_all_records': fetch_all_records,
    'get_readings_for_site': get_readings_for_site
})

dview.execute("""
import pandas as pd
import numpy as np
import requests
import itertools
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from tqdm import tqdm
import logging
logging.getLogger("cmdstanpy").setLevel(logging.ERROR)
""")

years = list(range(2008, 2025))
site_codes = ov25['site_code'].unique()

def load_site_data(site_code, years):
    return get_readings_for_site(site_code, years)

site_data = list(tqdm(
    dview.imap(load_site_data, site_codes, [years]*len(site_codes)),
    total=len(site_codes),
    desc="Loading site data"
))

Loading site data: 100%|██████████| 183/183 [01:47<00:00,  1.70it/s]


In [31]:
data_dict = dict(zip(site_codes, site_data))
dview.push({'data_dict': data_dict})

def process_site(site_code, years, cps, sps):
    try:
        df = data_dict.get(site_code)
        if df is None or df.empty:
            return None

        df['msmt_date'] = pd.to_datetime(df['msmt_date'])
        df = df[['msmt_date', 'wlm_rpe']].rename(columns={'msmt_date': 'ds', 'wlm_rpe': 'y'})
        df = df.sort_values('ds')

        model = Prophet(changepoint_prior_scale=cps,
                        seasonality_prior_scale=sps,
                        seasonality_mode='additive',
                        yearly_seasonality=True)
        model.fit(df)
        
        cv_results = cross_validation(model,
                                      initial='730 days',
                                      period='90 days',
                                      horizon='90 days',
                                      parallel="processes")
        metrics_df = performance_metrics(cv_results)
        avg_metrics = metrics_df.mean()
        
        y_full = df['y'].values
        scale_mase = np.mean(np.abs(np.diff(y_full)))
        scale_rmsse = np.mean((np.diff(y_full))**2)
        
        mae_cv = np.mean(np.abs(cv_results['y'] - cv_results['yhat']))
        mse_cv = np.mean((cv_results['y'] - cv_results['yhat'])**2)
        
        mase_val = mae_cv / scale_mase if scale_mase != 0 else np.nan
        rmsse_val = np.sqrt(mse_cv / scale_rmsse) if scale_rmsse != 0 else np.nan
        
        avg_metrics_dict = avg_metrics.to_dict()
        avg_metrics_dict['mase'] = mase_val
        avg_metrics_dict['rmsse'] = rmsse_val
        
        return avg_metrics_dict
    except Exception as e:
        return None

dview.push({'process_site': process_site})

param_grid = {
    'changepoint_prior_scale': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 0.5, 1.0, 10.0]
}

CHUNK_SIZE = 5

aggregated_results = []
grid = list(itertools.product(param_grid['changepoint_prior_scale'],
                              param_grid['seasonality_prior_scale']))

for cps, sps in grid:
    collected = {}
    n_valid = 0
    
    for start_idx in tqdm(range(0, len(site_codes), CHUNK_SIZE)):
        chunk = site_codes[start_idx : start_idx + CHUNK_SIZE]

        async_results = dview.map_async(
            process_site,
            chunk,
            [years] * len(chunk),
            [cps]   * len(chunk),
            [sps]   * len(chunk)
        )

        site_results = async_results.get()
        for res in site_results:
            if res is not None:
                n_valid += 1
                for key, value in res.items():
                    collected.setdefault(key, []).append(value)

    if n_valid > 0:
        avg_metrics = {key: np.mean(vals) for key, vals in collected.items()}
        avg_metrics.update({
            'changepoint_prior_scale': cps,
            'seasonality_prior_scale': sps,
            'seasonality_mode': 'additive',
            'n_sites': n_valid
        })
        aggregated_results.append(avg_metrics)
    else:
        print(f"No valid results for parameters: cps={cps}, sps={sps}")

results_df = pd.DataFrame(aggregated_results)
results_df = results_df.sort_values('mase')
results_df.to_csv("prophet_train.csv", index=False)

print("Aggregated Cross Validation Grid Search Results (averaged over sites):")
print(results_df)
print("\nBest Parameters based on aggregated MASE:")
print(results_df.iloc[0])

100%|██████████| 37/37 [08:51<00:00, 14.36s/it]

Aggregated Cross Validation Grid Search Results (averaged over sites):
                      horizon       mse      rmse       mae      mape  \
0  49 days 14:43:09.417757729  0.011480  0.038250  0.012461  0.000292   
1  49 days 14:43:09.417757729  0.011552  0.038475  0.012490  0.000293   
2  49 days 14:43:09.417757729  0.011573  0.038459  0.012465  0.000292   
3  49 days 14:43:09.417757729  0.011597  0.038505  0.012497  0.000293   
4  49 days 14:43:09.417757729  0.011606  0.038543  0.012501  0.000293   
5  49 days 14:43:09.417757729  0.011166  0.037792  0.011919  0.000275   
6  49 days 14:43:09.417757729  0.011455  0.038307  0.011953  0.000276   
7  49 days 14:43:09.417757729  0.011472  0.038326  0.011950  0.000276   
8  49 days 14:43:09.417757729  0.011466  0.038317  0.011954  0.000276   
9  49 days 14:43:09.417757729  0.011544  0.038420  0.011960  0.000277   
10 49 days 14:43:09.417757729  0.011124  0.037749  0.011833  0.000273   
11 49 days 14:43:09.417757729  0.011471  0.038371  0.

In [33]:
best_params = results_df.iloc[0]
best_cps = best_params['changepoint_prior_scale']
best_sps = best_params['seasonality_prior_scale']

def prophet_interpolation(site_code, cps, sps):
    try:
        df = data_dict.get(site_code)
        if df is None or df.empty:
            return None

        df['msmt_date'] = pd.to_datetime(df['msmt_date'])
        df = df[['msmt_date', 'wlm_rpe']].rename(columns={'msmt_date': 'ds', 'wlm_rpe': 'y'})
        df = df.sort_values('ds')

        model = Prophet(changepoint_prior_scale=cps,
                        seasonality_prior_scale=sps,
                        seasonality_mode='additive',
                        yearly_seasonality=True)
        model.fit(df)
        
        future = model.make_future_dataframe(periods=365)  # Adjust periods as needed
        forecast = model.predict(future)
        
        return forecast, df
    except Exception as e:
        print(f"Error processing site {site_code}: {e}")
        return None, None

def calculate_quarterly_error_metrics(forecast, df):
    # Merge forecast with actual data
    merged = pd.merge(df, forecast[['ds', 'yhat']], on='ds', how='inner')
    
    # Extract quarter from the date
    merged['quarter'] = merged['ds'].dt.quarter  # 1=Q1, 2=Q2, 3=Q3, 4=Q4
    
    # Initialize a dictionary to store quarterly metrics
    quarterly_metrics = {}
    
    for quarter in range(1, 5):
        quarter_data = merged[merged['quarter'] == quarter]
        
        if len(quarter_data) > 0:
            # Calculate error metrics for the quarter
            mae = np.mean(np.abs(quarter_data['y'] - quarter_data['yhat']))
            mse = np.mean((quarter_data['y'] - quarter_data['yhat'])**2)
            rmse = np.sqrt(mse)
            
            quarterly_metrics[f'q{quarter}_mae'] = mae
            quarterly_metrics[f'q{quarter}_mse'] = mse
            quarterly_metrics[f'q{quarter}_rmse'] = rmse
        else:
            # If no data for the quarter, set metrics to NaN
            quarterly_metrics[f'q{quarter}_mae'] = np.nan
            quarterly_metrics[f'q{quarter}_mse'] = np.nan
            quarterly_metrics[f'q{quarter}_rmse'] = np.nan
    
    return quarterly_metrics

all_site_metrics = []

for site_code in ov25:
    forecast, df = prophet_interpolation(site_code, best_cps, best_sps)
    if forecast is not None and df is not None:
        metrics = calculate_quarterly_error_metrics(forecast, df)
        metrics['site_code'] = site_code
        all_site_metrics.append(metrics)

# Convert to DataFrame for easier analysis
metrics_df = pd.DataFrame(all_site_metrics)
metrics_df.to_csv("prophet_interpolation_quarterly_metrics.csv", index=False)

print("Quarterly Error Metrics for Each Site:")
print(metrics_df)

TypeError: cannot unpack non-iterable NoneType object